In [1]:
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
import cv2
import numpy as np
import tensorflow as tf
from PIL import Image, ImageTk, ImageDraw, ImageFilter
import os
import threading
import json

class PlantIdentifierApp:
    def __init__(self, root):
        self.root = root
        self.root.title("MedLeaf PH - Philippine Medicinal Plants")
        
        # Get screen dimensions
        screen_width = self.root.winfo_screenwidth()
        screen_height = self.root.winfo_screenheight()
        
        # Calculate optimal window size (80% of screen, with min/max limits)
        min_width, min_height = 1000, 700
        max_width, max_height = 1600, 1200
        
        optimal_width = min(max(int(screen_width * 0.8), min_width), max_width)
        optimal_height = min(max(int(screen_height * 0.8), min_height), max_height)
        
        # Center the window on screen
        x = (screen_width - optimal_width) // 2
        y = (screen_height - optimal_height) // 2
        
        self.root.geometry(f"{optimal_width}x{optimal_height}+{x}+{y}")
        self.root.configure(bg='#f8fffe')
        self.root.resizable(True, True)
        
        # Set minimum window size
        self.root.minsize(min_width, min_height)
        
        # Store dimensions for responsive design
        self.window_width = optimal_width
        self.window_height = optimal_height
        self.is_large_screen = screen_width >= 1920  # For 1080p+ displays
        
        # Color scheme
        self.colors = {
            'primary': '#2E7D32',      # Deep green
            'secondary': '#4CAF50',     # Medium green
            'accent': '#81C784',        # Light green
            'warning': '#FF8F00',       # Orange
            'danger': '#F44336',        # Red
            'info': '#2196F3',          # Blue
            'light': '#E8F5E8',         # Very light green
            'white': '#FFFFFF',
            'gray_light': '#F5F5F5',
            'gray_medium': '#E0E0E0',
            'gray_dark': '#424242',
            'text_primary': '#212121',
            'text_secondary': '#757575'
        }
        
        # Initialize variables
        self.camera = None
        self.camera_running = False
        self.model = None
        self.class_names = []
        self.current_image = None
        
        # Confidence thresholds for leaf detection
        self.min_confidence_threshold = 15.0
        self.uncertainty_threshold = 25.0
        
        # Plant information database
        self.plant_info = self.load_plant_info()
        
        # Setup styles first
        self.setup_styles()
        
        # Setup UI
        self.setup_ui()
        
        # Load model
        self.load_model()
        
    def setup_styles(self):
        """Setup custom styles for ttk widgets"""
        style = ttk.Style()
        
        # Configure button styles
        style.configure("Primary.TButton",
                       background=self.colors['secondary'],
                       foreground='white',
                       borderwidth=0,
                       focuscolor='none',
                       padding=(20, 15))
        
        style.map("Primary.TButton",
                 background=[('active', self.colors['primary']),
                            ('pressed', self.colors['primary'])])
        
        style.configure("Secondary.TButton",
                       background=self.colors['warning'],
                       foreground='white',
                       borderwidth=0,
                       focuscolor='none',
                       padding=(20, 15))
        
        style.map("Secondary.TButton",
                 background=[('active', '#F57C00'),
                            ('pressed', '#F57C00')])
        
    def setup_ui(self):
        # Main container with gradient-like effect
        main_container = tk.Frame(self.root, bg=self.colors['white'])
        main_container.pack(fill='both', expand=True)
        
        # Header with modern design
        self.setup_header(main_container)
        
        # Create main scrollable content area
        self.setup_scrollable_area(main_container)
        
        # Setup camera window (hidden initially)
        self.setup_camera_window()
        
    def setup_header(self, parent):
        """Setup modern header with left-aligned title and icon - thinner header"""
        # Much thinner header height
        header_height = 70 if self.is_large_screen else 60
        
        header_frame = tk.Frame(parent, bg=self.colors['primary'], height=header_height)
        header_frame.pack(fill='x')
        header_frame.pack_propagate(False)
        
        # Create gradient effect using multiple frames
        gradient_frame = tk.Frame(header_frame, bg=self.colors['primary'])
        gradient_frame.pack(fill='both', expand=True)
        
        # Header content container - pushed more to the left
        header_content = tk.Frame(gradient_frame, bg=self.colors['primary'])
        header_content.pack(fill='both', expand=True, padx=20, pady=8)
        
        # Title and icon container - horizontal layout
        title_container = tk.Frame(header_content, bg=self.colors['primary'])
        title_container.pack(anchor='w', pady=(5, 2))
        
        # Responsive font sizes - slightly smaller for thinner header
        logo_size = 28 if self.is_large_screen else 24
        title_size = 26 if self.is_large_screen else 22
        subtitle_size = 13 if self.is_large_screen else 11
        
        # Icon and title side by side with Roboto font
        logo_label = tk.Label(title_container, text="🌿", font=('Arial', logo_size), 
                             bg=self.colors['primary'], fg='white')
        logo_label.pack(side='left', padx=(0, 12))
        
        title_label = tk.Label(title_container, text="MedLeaf PH", 
                              font=('Roboto', title_size, 'bold'), fg='white', bg=self.colors['primary'])
        title_label.pack(side='left')
        
        # Subtitle below the title with Roboto font
        subtitle_label = tk.Label(header_content, text="AI-Powered Philippine Medicinal Plant Identifier", 
                                 font=('Roboto', subtitle_size), fg='#C8E6C9', bg=self.colors['primary'])
        subtitle_label.pack(anchor='w', pady=(2, 0))
        
    def setup_scrollable_area(self, parent):
        """Setup improved scrollable content area - responsive padding"""
        # Responsive padding based on screen size
        side_padding = 40 if self.is_large_screen else 20
        bottom_padding = 30 if self.is_large_screen else 20
        
        # Content container with shadow effect
        content_container = tk.Frame(parent, bg=self.colors['gray_light'], relief='flat')
        content_container.pack(fill='both', expand=True, padx=side_padding, pady=(0, bottom_padding))
        
        # Create canvas and scrollbar with modern styling
        canvas_frame = tk.Frame(content_container, bg=self.colors['white'], relief='solid', bd=1)
        canvas_frame.pack(fill='both', expand=True)
        
        self.canvas = tk.Canvas(canvas_frame, bg=self.colors['white'], highlightthickness=0,
                               borderwidth=0, relief='flat')
        
        # Custom scrollbar
        scrollbar = ttk.Scrollbar(canvas_frame, orient="vertical", command=self.canvas.yview)
        self.scrollable_frame = tk.Frame(self.canvas, bg=self.colors['white'])
        
        # Configure scrollable frame
        self.scrollable_frame.bind(
            "<Configure>",
            lambda e: self.canvas.configure(scrollregion=self.canvas.bbox("all"))
        )
        
        self.canvas_window = self.canvas.create_window((0, 0), window=self.scrollable_frame, anchor="n")
        self.canvas.configure(yscrollcommand=scrollbar.set)
        
        # Pack canvas and scrollbar
        self.canvas.pack(side="left", fill="both", expand=True)
        scrollbar.pack(side="right", fill="y")
        
        # Bind events
        self.canvas.bind("<MouseWheel>", self._on_mousewheel)
        self.canvas.bind("<Configure>", self._on_canvas_configure)
        self.root.bind("<MouseWheel>", self._on_mousewheel)
        
        # Setup main content in scrollable area
        self.setup_main_content()
        
    def setup_main_content(self):
        """Setup the main content area with modern cards - responsive layout"""
        # Responsive content padding based on window size
        content_padding = 80 if self.is_large_screen else 60
        vertical_padding = 40 if self.is_large_screen else 30
        
        # Main content with max width and centering
        main_frame = tk.Frame(self.scrollable_frame, bg=self.colors['white'])
        main_frame.pack(expand=True, fill='both', padx=content_padding, pady=vertical_padding)
        
        # Image display card with modern styling
        self.setup_image_card(main_frame)
        
        # Action buttons with modern design
        self.setup_action_buttons(main_frame)
        
        # Information card
        self.setup_info_card(main_frame)
        
        # Results area
        self.results_frame = tk.Frame(main_frame, bg=self.colors['white'])
        self.results_frame.pack(expand=True, pady=(vertical_padding, 0), fill='both')
        
    def setup_image_card(self, parent):
        """Setup modern image display card - responsive sizing"""
        # Card container with shadow effect
        image_card = tk.Frame(parent, bg=self.colors['white'], relief='solid', bd=1)
        image_card.pack(pady=(0, 30), fill='x')
        
        # Responsive header height
        header_height = 60 if self.is_large_screen else 50
        
        # Card header
        card_header = tk.Frame(image_card, bg=self.colors['light'], height=header_height)
        card_header.pack(fill='x')
        card_header.pack_propagate(False)
        
        # Responsive font size
        header_font_size = 18 if self.is_large_screen else 16
        
        header_label = tk.Label(card_header, text="📸 Plant Image", 
                               font=('Arial', header_font_size, 'bold'), 
                               fg=self.colors['primary'], bg=self.colors['light'])
        header_label.pack(expand=True)
        
        # Image display area with modern styling
        self.image_frame = tk.Frame(image_card, bg='#FAFAFA', relief='flat')
        self.image_frame.pack(pady=25 if self.is_large_screen else 20, 
                             padx=25 if self.is_large_screen else 20, fill='x')
        
        # Placeholder with modern design - responsive sizing
        placeholder_padding = 25 if self.is_large_screen else 20
        
        placeholder_frame = tk.Frame(self.image_frame, bg='#F0F8F0', relief='solid', bd=2,
                                    highlightbackground=self.colors['accent'], highlightthickness=2)
        placeholder_frame.pack(pady=placeholder_padding, padx=placeholder_padding)
        
        # Responsive emoji and padding
        emoji_size = 100 if self.is_large_screen else 80
        emoji_padding_y = 80 if self.is_large_screen else 60
        emoji_padding_x = 150 if self.is_large_screen else 120
        
        self.image_label = tk.Label(placeholder_frame, text="🍃", font=('Arial', emoji_size), 
                                   bg='#F0F8F0', fg=self.colors['secondary'])
        self.image_label.pack(pady=emoji_padding_y, padx=emoji_padding_x)
        
        # Image info label - responsive font
        info_font_size = 14 if self.is_large_screen else 12
        info_padding = 25 if self.is_large_screen else 20
        
        info_label = tk.Label(self.image_frame, 
                             text="Upload an image or use camera to identify medicinal plants",
                             font=('Arial', info_font_size), fg=self.colors['text_secondary'], bg='#FAFAFA')
        info_label.pack(pady=(0, info_padding))
        
    def setup_action_buttons(self, parent):
        """Setup modern action buttons - responsive sizing"""
        button_card = tk.Frame(parent, bg=self.colors['white'])
        button_card.pack(pady=(0, 30))
        
        # Button container with centered layout
        button_container = tk.Frame(button_card, bg=self.colors['white'])
        button_container.pack()
        
        # Responsive button sizing
        button_font_size = 16 if self.is_large_screen else 14
        button_width = 18 if self.is_large_screen else 15
        button_pady = 15 if self.is_large_screen else 12
        button_spacing = 20 if self.is_large_screen else 15
        
        # Modern camera button with responsive styling
        self.camera_btn = tk.Button(button_container, 
                                   text="📷  Open Camera", 
                                   font=('Arial', button_font_size, 'bold'),
                                   bg=self.colors['secondary'], 
                                   fg='white', 
                                   relief='flat',
                                   borderwidth=0,
                                   command=self.open_camera, 
                                   cursor='hand2', 
                                   width=button_width,
                                   pady=button_pady,
                                   activebackground=self.colors['primary'],
                                   activeforeground='white')
        self.camera_btn.pack(side='left', padx=(0, button_spacing), pady=10)
        
        # Modern upload button
        self.gallery_btn = tk.Button(button_container, 
                                    text="📁  Upload Image", 
                                    font=('Arial', button_font_size, 'bold'),
                                    bg=self.colors['warning'], 
                                    fg='white', 
                                    relief='flat',
                                    borderwidth=0,
                                    command=self.open_gallery, 
                                    cursor='hand2', 
                                    width=button_width,
                                    pady=button_pady,
                                    activebackground='#F57C00',
                                    activeforeground='white')
        self.gallery_btn.pack(side='left', padx=(button_spacing, 0), pady=10)
        
    def setup_info_card(self, parent):
        """Setup modern information card - fixed emoji layout"""
        info_card = tk.Frame(parent, bg=self.colors['white'], relief='solid', bd=1)
        info_card.pack(pady=(0, 30), fill='x')
        
        # Card header with icon
        info_header = tk.Frame(info_card, bg=self.colors['info'], height=50)
        info_header.pack(fill='x')
        info_header.pack_propagate(False)
        
        header_label = tk.Label(info_header, text="🤖 How AI Identification Works", 
                               font=('Arial', 16, 'bold'), 
                               fg='white', bg=self.colors['info'])
        header_label.pack(expand=True)
        
        # Content area with better spacing
        content_frame = tk.Frame(info_card, bg=self.colors['white'])
        content_frame.pack(pady=25, padx=30, fill='x')
        
        # Steps with modern styling - removed emoji boxes
        steps = [
            "Capture or upload a clear plant leaf image",
            "AI analyzes leaf features using deep learning", 
            "Get instant results with confidence scores",
            "Learn about medicinal properties and uses",
            "System validates if image contains plant material"
        ]
        
        for i, step in enumerate(steps, 1):
            step_frame = tk.Frame(content_frame, bg=self.colors['white'])
            step_frame.pack(fill='x', pady=8)
            
            # Simple number instead of emoji
            number_label = tk.Label(step_frame, text=f"{i}.", font=('Arial', 14, 'bold'), 
                                   bg=self.colors['white'], fg=self.colors['primary'])
            number_label.pack(side='left', padx=(0, 15))
            
            step_label = tk.Label(step_frame, text=step, font=('Arial', 12), 
                                 bg=self.colors['white'], fg=self.colors['text_primary'],
                                 anchor='w')
            step_label.pack(side='left', fill='x', expand=True)
        
    def setup_camera_window(self):
        """Setup modern camera window - responsive sizing"""
        self.camera_frame = tk.Toplevel(self.root)
        self.camera_frame.withdraw()
        self.camera_frame.title("MedLeaf PH - Camera")
        
        # Responsive camera window size
        cam_width = 800 if self.is_large_screen else 680
        cam_height = 720 if self.is_large_screen else 620
        
        # Center camera window
        screen_width = self.root.winfo_screenwidth()
        screen_height = self.root.winfo_screenheight()
        cam_x = (screen_width - cam_width) // 2
        cam_y = (screen_height - cam_height) // 2
        
        self.camera_frame.geometry(f"{cam_width}x{cam_height}+{cam_x}+{cam_y}")
        self.camera_frame.configure(bg=self.colors['white'])
        self.camera_frame.protocol("WM_DELETE_WINDOW", self.close_camera)
        
        # Responsive header height and font
        header_height = 80 if self.is_large_screen else 60
        title_font_size = 20 if self.is_large_screen else 18
        
        # Camera header
        cam_header = tk.Frame(self.camera_frame, bg=self.colors['primary'], height=header_height)
        cam_header.pack(fill='x')
        cam_header.pack_propagate(False)
        
        cam_title = tk.Label(cam_header, text="📷 Camera View", 
                            font=('Arial', title_font_size, 'bold'), 
                            fg='white', bg=self.colors['primary'])
        cam_title.pack(expand=True)
        
        # Camera display with border - responsive padding
        cam_padding = 30 if self.is_large_screen else 20
        
        cam_container = tk.Frame(self.camera_frame, bg=self.colors['gray_light'])
        cam_container.pack(pady=cam_padding, padx=cam_padding, fill='both', expand=True)
        
        self.camera_label = tk.Label(cam_container, bg=self.colors['white'], 
                                    relief='solid', bd=2)
        self.camera_label.pack(pady=15 if self.is_large_screen else 10, 
                              padx=15 if self.is_large_screen else 10, 
                              fill='both', expand=True)
        
        # Modern camera controls - responsive sizing
        button_font_size = 16 if self.is_large_screen else 14
        button_pady = 12 if self.is_large_screen else 10
        button_padx = 25 if self.is_large_screen else 20
        
        cam_controls = tk.Frame(self.camera_frame, bg=self.colors['white'])
        cam_controls.pack(pady=(0, cam_padding))
        
        capture_btn = tk.Button(cam_controls, text="📸 Capture Photo", 
                               font=('Arial', button_font_size, 'bold'),
                               bg=self.colors['secondary'], fg='white', 
                               relief='flat', borderwidth=0,
                               command=self.capture_image, cursor='hand2',
                               pady=button_pady, padx=button_padx,
                               activebackground=self.colors['primary'])
        capture_btn.pack(side='left', padx=10)
        
        close_btn = tk.Button(cam_controls, text="❌ Close Camera", 
                             font=('Arial', button_font_size, 'bold'),
                             bg=self.colors['danger'], fg='white', 
                             relief='flat', borderwidth=0,
                             command=self.close_camera, cursor='hand2',
                             pady=button_pady, padx=button_padx,
                             activebackground='#D32F2F')
        close_btn.pack(side='left', padx=10)
        
    def load_plant_info(self):
        """Load plant information database with all 40 Philippine medicinal plants"""
        return {
            "1Hibiscus rosa-sinensis(HRS)": {
            "scientific_name": "Hibiscus rosa-sinensis",
            "tagalog_name": "Gumamela",
            "common_names": ["Gumamela", "Chinese Hibiscus", "Shoe Flower"],
            "medicinal_uses": ["Ubo", "Lagnat", "Sakit sa balat", "Parang bawas", "Pamamaga ng lalamunan"],
            "preparation": "Pakuluin ang mga bulaklak para sa tsaa o gumawa ng poultice para sa balat",
            "dosage": "1 baso ng pinaglagang bulaklak, 3 beses sa isang araw",
            "parts_used": ["Bulaklak", "Dahon"]
        },
        "2Psidium guajava(PG)": {
            "scientific_name": "Psidium guajava",
            "tagalog_name": "Bayabas",
            "common_names": ["Bayabas", "Guava", "Guayaba"],
            "medicinal_uses": ["Pagtatae", "Sugat", "Sakit ng ngipin", "Mumog sa lalamunan", "Impeksyon sa balat"],
            "preparation": "Ngumunyain ang mga dahon o pakuluin para sa tubig pambubunga",
            "dosage": "1/2 baso ng tsaa ng dahon, 3 beses sa isang araw",
            "parts_used": ["Dahon", "Bunga", "Balat ng puno"]
        },
        "3Antidesma bunius(AB)": {
            "scientific_name": "Antidesma bunius",
            "tagalog_name": "Bignay",
            "common_names": ["Bignay", "Chinese Laurel", "Currant Tree"],
            "medicinal_uses": ["Diabetes", "Mataas na presyon", "Sakit sa bato", "Panlinis ng dugo"],
            "preparation": "Pakuluin ang mga dahon para sa tsaa",
            "dosage": "1 baso ng tsaa, 2 beses sa isang araw",
            "parts_used": ["Dahon", "Bunga", "Balat ng puno"]
        },
        "4Vitex negundo(VN)": {
            "scientific_name": "Vitex negundo",
            "tagalog_name": "Lagundi",
            "common_names": ["Lagundi", "Five-leaved chaste tree", "Vitex"],
            "medicinal_uses": ["Ubo", "Hika", "Lagnat", "Sakit ng ulo", "Tigdas", "Galis-aso"],
            "preparation": "Pakuluin ang 1 kinis ng mga dahon sa 3 basong tubig ng 10 minuto",
            "dosage": "Sanggol: 1 kutsarita 3x; 7-12 taon: 1/2 baso 3x; Matanda: 1 baso 3x",
            "parts_used": ["Dahon", "Bulaklak", "Sanga"]
        },
        "5Moringa oleifera(MO)": {
            "scientific_name": "Moringa oleifera",
            "tagalog_name": "Malunggay",
            "common_names": ["Malunggay", "Drumstick tree", "Miracle tree"],
            "medicinal_uses": ["Kulang sa nutrisyon", "Diabetes", "Pamamaga", "Pagdagdag ng gatas ng ina"],
            "preparation": "Kainin ang sariwang dahon o gawing tsaa",
            "dosage": "1-2 kutsarang dahon araw-araw sa pagkain",
            "parts_used": ["Dahon", "Bunga", "Buto", "Ugat"]
        },
        "6Blumea balsamifera(BB)": {
            "scientific_name": "Blumea balsamifera",
            "tagalog_name": "Sambong",
            "common_names": ["Sambong", "Ngai camphor", "Blumea"],
            "medicinal_uses": ["Bato sa bato", "Mataas na presyon", "Ubo", "Hirap umiihi", "Rayuma"],
            "preparation": "Pakuluin ang tuyo at sariwang dahon",
            "dosage": "1 baso ng tsaa, 3 beses sa isang araw",
            "parts_used": ["Dahon", "Buong halaman"]
        },
        "7Origanum vulgare(OV)": {
            "scientific_name": "Origanum vulgare",
            "tagalog_name": "Oregano",
            "common_names": ["Oregano", "Wild marjoram", "Origanum"],
            "medicinal_uses": ["Ubo", "Sipon", "Hika", "Sakit sa balat", "Pangdisimpeyte"],
            "preparation": "Gawing tsaa ang mga dahon o gamitin sa pagluluto",
            "dosage": "1 kutsarita ng dahon sa tubig, 2-3 beses sa araw",
            "parts_used": ["Dahon", "Bulaklak"]
        },
        "8Pepromia pellucida(PP)": {
            "scientific_name": "Peperomia pellucida",
            "tagalog_name": "Pansit-pansitan",
            "common_names": ["Pansit-pansitan", "Ulasimang bato", "Shiny bush"],
            "medicinal_uses": ["Rayuma", "Sakit sa bato", "Sakit sa balat", "Mataas na asukal"],
            "preparation": "Kainin nang hilaw o pakuluin para sa tsaa",
            "dosage": "1 dakot ng sariwang halaman araw-araw",
            "parts_used": ["Buong halaman", "Dahon"]
        },
        "9Centella asiatica(CA)": {
            "scientific_name": "Centella asiatica",
            "tagalog_name": "Takip-kuhol",
            "common_names": ["Takip-kuhol", "Gotu kola", "Centella"],
            "medicinal_uses": ["Sugat", "Pagpapahusay ng isip", "Paggaling ng balat", "Rayuma"],
            "preparation": "Ilagay ang sariwang dahon sa sugat o gawing tsaa",
            "dosage": "Ilahok sa salad o tsaa araw-araw",
            "parts_used": ["Dahon", "Buong halaman"]
        },
        "10Coleus scutellarioides(CS)": {
            "scientific_name": "Coleus scutellarioides",
            "tagalog_name": "Mayana",
            "common_names": ["Mayana", "Painted Nettle", "Coleus"],
            "medicinal_uses": ["Sakit sa balat", "Sugat", "Insektong kagat", "Pamamaga"],
            "preparation": "Durugin ang dahon at ilagay sa apektadong bahagi",
            "dosage": "Gamitin bilang poultice 2-3 beses sa araw",
            "parts_used": ["Dahon"]
        },
        "11Phyllanthus niruri(PN)": {
            "scientific_name": "Phyllanthus niruri",
            "tagalog_name": "Sampa-sampalukan",
            "common_names": ["Sampa-sampalukan", "Stone-breaker", "Chanca piedra"],
            "medicinal_uses": ["Bato sa bato", "Sakit sa atay", "Diabetes", "Sakit sa tiyan"],
            "preparation": "Pakuluin ang buong halaman",
            "dosage": "1/2 baso ng tsaa, 2 beses sa araw",
            "parts_used": ["Buong halaman", "Dahon"]
        },
        "12Corchorus olitorius(CO)": {
            "scientific_name": "Corchorus olitorius",
            "tagalog_name": "Saluyot",
            "common_names": ["Saluyot", "Jute leaves", "Corchorus"],
            "medicinal_uses": ["Konstipasyon", "Pamamaga", "Sakit sa bato", "Mataas na kolesterol"],
            "preparation": "Lutuin bilang gulay o gawing tsaa",
            "dosage": "Kainin bilang gulay o uminom ng 1 baso ng tsaa",
            "parts_used": ["Dahon", "Sanga"]
        },
        "13Momordica charantia(MC)": {
            "scientific_name": "Momordica charantia",
            "tagalog_name": "Ampalaya",
            "common_names": ["Ampalaya", "Bitter gourd", "Bitter melon"],
            "medicinal_uses": ["Diabetes", "Sakit sa tiyan", "Bulate", "Panlinis ng dugo"],
            "preparation": "Lutuin bilang gulay o uminom ng sariwang katas",
            "dosage": "1/4 baso ng katas bago kumain o kainin bilang gulay",
            "parts_used": ["Bunga", "Dahon"]
        },
        "14Euphorbia hirta(EH)": {
            "scientific_name": "Euphorbia hirta",
            "tagalog_name": "Tawa-tawa",
            "common_names": ["Tawa-tawa", "Asthma plant", "Snake weed"],
            "medicinal_uses": ["Hika", "Ubo", "Pagtatae", "Dengue (tradisyonal)"],
            "preparation": "Pakuluin ang buong halaman",
            "dosage": "1/2 baso ng tsaa, 3 beses sa araw",
            "parts_used": ["Buong halaman"]
        },
        "15Curcuma longa(CL)": {
            "scientific_name": "Curcuma longa",
            "tagalog_name": "Luyang dilaw",
            "common_names": ["Luyang dilaw", "Turmeric", "Yellow ginger"],
            "medicinal_uses": ["Pamamaga", "Rayuma", "Sakit sa tiyan", "Sakit sa lalamunan"],
            "preparation": "Gamitin ang sariwang ugat o powder sa pagluluto",
            "dosage": "1 kutsarita ng powder sa gatas o tsaa",
            "parts_used": ["Ugat", "Rhizome"]
        },
        "16Carmona retusa(CR)": {
            "scientific_name": "Carmona retusa",
            "tagalog_name": "Tsaang gubat",
            "common_names": ["Tsaang gubat", "Philippine Tea", "Forest tea"],
            "medicinal_uses": ["Sakit sa tiyan", "Pagtatae", "Sakit ng ngipin"],
            "preparation": "Pakuluin ang dahon hanggang maging kalahati ang tubig",
            "dosage": "7-12 taon: 1½ kutsara; 13+ taon: 3 kutsara ng dahon",
            "parts_used": ["Dahon"]
        },
        "17Senna alata(SA)": {
            "scientific_name": "Senna alata",
            "tagalog_name": "Akapulko",
            "common_names": ["Akapulko", "Ringworm bush", "Senna"],
            "medicinal_uses": ["Hadhad", "Galis", "Buni", "Impeksyon sa balat"],
            "preparation": "Durugin ang dahon at ilagay sa apektadong bahagi",
            "dosage": "Gamitin bilang ointment 2-3 beses sa araw",
            "parts_used": ["Dahon", "Bulaklak"]
        },
        "18Mentha cordifolia Opiz(MCO)": {
            "scientific_name": "Mentha cordifolia",
            "tagalog_name": "Yerba buena",
            "common_names": ["Yerba buena", "Spearmint", "Mint"],
            "medicinal_uses": ["Sakit ng ulo", "Sipon", "Sakit sa tiyan", "Nerbiyos"],
            "preparation": "Gawing tsaa ang dahon o amoy-amuyin",
            "dosage": "1 baso ng tsaa o ilagay sa noo para sa sakit ng ulo",
            "parts_used": ["Dahon"]
        },
        "19Capsicum frutescens(CF)": {
            "scientific_name": "Capsicum frutescens",
            "tagalog_name": "Siling labuyo",
            "common_names": ["Siling labuyo", "Chili pepper", "Bird's eye chili"],
            "medicinal_uses": ["Rayuma", "Sakit ng ulo", "Pamparegla", "Panggising"],
            "preparation": "Gamitin sa pagluluto o gawing liniment",
            "dosage": "Konting dami lang sa pagkain o sa liniment",
            "parts_used": ["Bunga"]
        },
        "20Jatropha curcas(JC)": {
            "scientific_name": "Jatropha curcas",
            "tagalog_name": "Kasla",
            "common_names": ["Kasla", "Physic Nut", "Jatropha"],
            "medicinal_uses": ["Konstipasyon", "Sugat", "Sakit sa balat", "Tumor"],
            "preparation": "Gumamit ng dahon para sa poultice, buto para sa laxative",
            "dosage": "1-2 buto para sa konstipasyon (ingat: nakalalason)",
            "parts_used": ["Dahon", "Buto", "Sap"]
        },
        "21Ocimum basilicum(OB)": {
            "scientific_name": "Ocimum basilicum",
            "tagalog_name": "Balanoy",
            "common_names": ["Balanoy", "Sweet basil", "Holy basil"],
            "medicinal_uses": ["Ubo", "Sipon", "Sakit ng ulo", "Stress", "Lagnat"],
            "preparation": "Gawing tsaa ang dahon o kainin sa salad",
            "dosage": "1 baso ng tsaa o 5-10 dahon sa salad",
            "parts_used": ["Dahon", "Bulaklak"]
        },
        "22Nerium oleander(NO)": {
            "scientific_name": "Nerium oleander",
            "tagalog_name": "Adelfa",
            "common_names": ["Adelfa", "Oleander", "Rose bay"],
            "medicinal_uses": ["Sakit sa puso (tradisyonal)", "Rayuma", "Tumor"],
            "preparation": "BABALA: Nakalalason - hindi dapat gamitin nang walang eksperto",
            "dosage": "HUWAG GAMITIN - Delikadong halaman",
            "parts_used": ["NONE - Poisonous"]
        },
        "23Pandanus amaryllifolius(PA)": {
            "scientific_name": "Pandanus amaryllifolius",
            "tagalog_name": "Pandan",
            "common_names": ["Pandan", "Vanilla grass", "Screwpine"],
            "medicinal_uses": ["Sakit ng ulo", "Stress", "Pamparelaks", "Pampasarap ng pagkain"],
            "preparation": "Gawing tsaa ang dahon o gamitin sa pagluluto",
            "dosage": "1-2 dahon sa tsaa o sa pagkain",
            "parts_used": ["Dahon"]
        },
        "24Aloe barbadensis Miller(ABM)": {
            "scientific_name": "Aloe barbadensis miller",
            "tagalog_name": "Sabila",
            "common_names": ["Sabila", "Aloe vera", "Sabila"],
            "medicinal_uses": ["Paso", "Sugat", "Sakit sa balat", "Sakit sa tiyan"],
            "preparation": "Ilagay ang gel direkta sa balat o uminom ng katas",
            "dosage": "Direktang gel sa balat o 1-2 kutsara ng katas",
            "parts_used": ["Gel", "Dahon"]
        },
        "25Lagerstroemia speciosa(LS)": {
            "scientific_name": "Lagerstroemia speciosa",
            "tagalog_name": "Banaba",
            "common_names": ["Banaba", "Crape Myrtle", "Pride of India"],
            "medicinal_uses": ["Diabetes", "Mataas na presyon", "Sakit sa bato", "Pangtanggal ng timbang"],
            "preparation": "Pakuluin ang mga dahon para sa tsaa",
            "dosage": "1 baso ng tsaa bago kumain",
            "parts_used": ["Dahon", "Balat ng puno"]
        },
        "26Averrhoea bilimbi(AVB)": {
            "scientific_name": "Averrhoa bilimbi",
            "tagalog_name": "Kamias",
            "common_names": ["Kamias", "Cucumber tree", "Bilimbi"],
            "medicinal_uses": ["Mataas na presyon", "Diabetes", "Ubo", "Sakit sa balat"],
            "preparation": "Kainin ang bunga o gawing tsaa ang dahon",
            "dosage": "2-3 bunga o 1 baso ng tsaa ng dahon",
            "parts_used": ["Bunga", "Dahon"]
        },
        "27Annona muricata(AM)": {
            "scientific_name": "Annona muricata",
            "tagalog_name": "Guyabano",
            "common_names": ["Guyabano", "Soursop", "Graviola"],
            "medicinal_uses": ["Cancer (tradisyonal)", "Impeksyon", "Parasito", "Lagnat"],
            "preparation": "Gawing tsaa ang dahon o kainin ang bunga",
            "dosage": "1 baso ng tsaa ng dahon o 1/2 bunga",
            "parts_used": ["Dahon", "Bunga", "Buto"]
        },
        "28Citrus aurantiifolia(CIA)": {
            "scientific_name": "Citrus aurantiifolia",
            "tagalog_name": "Dayap",
            "common_names": ["Dayap", "Lime", "Key lime"],
            "medicinal_uses": ["Sipon", "Ubo", "Sakit sa lalamunan", "Pangtanggal ng amoy"],
            "preparation": "Gamitin ang katas sa tubig o tsaa",
            "dosage": "Katas ng 1 dayap sa mainit na tubig",
            "parts_used": ["Bunga", "Katas"]
        },
        "29Premna odorata(PO)": {
            "scientific_name": "Premna odorata",
            "tagalog_name": "Aduon",
            "common_names": ["Aduon", "Fragrant Premna", "Premna"],
            "medicinal_uses": ["Rayuma", "Sakit sa buto", "Pamamaga", "Sugat"],
            "preparation": "Pakuluin ang dahon para sa poultice o tsaa",
            "dosage": "Gamitin bilang poultice o 1/2 baso ng tsaa",
            "parts_used": ["Dahon", "Balat ng puno"]
        },
        "30Gliricidia sepium(GS)": {
            "scientific_name": "Gliricidia sepium",
            "tagalog_name": "Kakawati",
            "common_names": ["Kakawati", "Mexican lilac", "Gliricidia"],
            "medicinal_uses": ["Sugat", "Sakit sa balat", "Insektong kagat", "Tumor"],
            "preparation": "Durugin ang dahon para sa poultice",
            "dosage": "Ilagay sa apektadong bahagi 2-3 beses sa araw",
            "parts_used": ["Dahon", "Balat ng puno"]
        },
        "31Citrus sinensis(CIS)": {
            "scientific_name": "Citrus sinensis",
            "tagalog_name": "Dalandan",
            "common_names": ["Dalandan", "Sweet Orange", "Orange"],
            "medicinal_uses": ["Sipon", "Kulang sa Vitamin C", "Sakit sa lalamunan", "Stress"],
            "preparation": "Kainin ang bunga o gawing juice",
            "dosage": "1-2 bunga o 1 baso ng katas araw-araw",
            "parts_used": ["Bunga", "Balat", "Dahon"]
        },
        "32Mangifera indica(MI)": {
            "scientific_name": "Mangifera indica",
            "tagalog_name": "Mangga",
            "common_names": ["Mangga", "Mango", "Indian mango"],
            "medicinal_uses": ["Pagtatae", "Sakit sa tiyan", "Ubo", "Sakit sa lalamunan"],
            "preparation": "Gawing tsaa ang dahon o kainin ang bunga",
            "dosage": "1 baso ng tsaa ng dahon o 1 bunga",
            "parts_used": ["Dahon", "Bunga", "Balat ng puno"]
        },
        "33Citrus microcarpa(CM)": {
            "scientific_name": "Citrus microcarpa",
            "tagalog_name": "Kalamansi",
            "common_names": ["Kalamansi", "Philippine lime", "Calamondin"],
            "medicinal_uses": ["Sipon", "Ubo", "Pampapayat", "Panglinis ng dugo"],
            "preparation": "Gamitin ang katas sa tubig o sa pagkain",
            "dosage": "Katas ng 2-3 kalamansi sa mainit na tubig",
            "parts_used": ["Bunga", "Katas", "Dahon"]
        },
        "34Impatiens balsamina(IB)": {
            "scientific_name": "Impatiens balsamina",
            "tagalog_name": "Kamantigue",
            "common_names": ["Kamantigue", "Garden balsam", "Touch-me-not"],
            "medicinal_uses": ["Sugat", "Sakit sa balat", "Pamamaga", "Bukol"],
            "preparation": "Durugin ang dahon para sa poultice",
            "dosage": "Ilagay sa apektadong bahagi 2-3 beses sa araw",
            "parts_used": ["Dahon", "Bulaklak"]
        },
        "35Arachis hypogaea(AH)": {
            "scientific_name": "Arachis hypogaea",
            "tagalog_name": "Mani",
            "common_names": ["Mani", "Peanut", "Groundnut"],
            "medicinal_uses": ["Kulang sa protina", "Sakit sa balat", "Ubo", "Panlakas"],
            "preparation": "Kainin ang buto o gawing oil ang dahon",
            "dosage": "1 dakot ng mani o oil sa balat",
            "parts_used": ["Buto", "Dahon", "Oil"]
        },
        "36Tamarindus indica(TI)": {
            "scientific_name": "Tamarindus indica",
            "tagalog_name": "Sampalok",
            "common_names": ["Sampalok", "Tamarind", "Indian date"],
            "medicinal_uses": ["Konstipasyon", "Lagnat", "Sakit sa tiyan", "Diabetes"],
            "preparation": "Kainin ang bunga o gawing tsaa ang dahon",
            "dosage": "3-5 bunga o 1 baso ng tsaa ng dahon",
            "parts_used": ["Bunga", "Dahon", "Balat ng puno"]
        },
        "37Leucaena leucocephala(LL)": {
            "scientific_name": "Leucaena leucocephala",
            "tagalog_name": "Ipil-ipil",
            "common_names": ["Ipil-ipil", "River Tamarind", "Lead tree"],
            "medicinal_uses": ["Sugat", "Sakit sa balat", "Panglinis ng dugo", "Sakit sa buto"],
            "preparation": "Gawing poultice ang dahon o tsaa",
            "dosage": "Poultice 2x araw o 1/2 baso ng tsaa",
            "parts_used": ["Dahon", "Balat ng puno"]
        },
        "38Ipomoea batatas(IPB)": {
            "scientific_name": "Ipomoea batatas",
            "tagalog_name": "Kamote",
            "common_names": ["Kamote", "Sweet potato", "Camote"],
            "medicinal_uses": ["Diabetes", "Sakit sa tiyan", "Kulang sa nutrisyon", "Konstipasyon"],
            "preparation": "Lutuin ang ugat o gawing tsaa ang dahon",
            "dosage": "1 piraso ng kamote o 1 baso ng tsaa ng dahon",
            "parts_used": ["Ugat", "Dahon"]
        },
        "39Manihot esculenta(ME)": {
            "scientific_name": "Manihot esculenta",
            "tagalog_name": "Kamoteng kahoy",
            "common_names": ["Kamoteng kahoy", "Cassava", "Yuca"],
            "medicinal_uses": ["Gutom", "Kulang sa carbohydrates", "Sakit sa tiyan", "Diarrhea"],
            "preparation": "Lutuin ang ugat o gawing starch",
            "dosage": "1-2 piraso ng luto o 1 kutsara ng starch",
            "parts_used": ["Ugat", "Dahon (konti lang)"]
        },
        "40Citrus maxima(CMA)": {
            "scientific_name": "Citrus maxima",
            "tagalog_name": "Suha",
            "common_names": ["Suha", "Pomelo", "Grapefruit"],
            "medicinal_uses": ["Mataas na presyon", "Diabetes", "Sakit sa lalamunan", "Stress"],
            "preparation": "Kainin ang bunga o gawing tsaa ang dahon",
            "dosage": "1/4 ng bunga o 1 baso ng tsaa ng dahon",
            "parts_used": ["Bunga", "Dahon", "Balat"]
            }
        }
    
    def load_model(self):
        """Load the Keras model and class names"""
        try:
            model_path = "philippine_medicinal_plants_cnn_new.keras"
            
            if os.path.exists(model_path):
                print(f"Loading model from: {model_path}")
                self.model = tf.keras.models.load_model(model_path)
                print("Model loaded successfully!")
            else:
                messagebox.showerror("Model Not Found", 
                    f"Model file not found: {model_path}\n"
                    f"Please ensure the file exists in the same directory.")
                return
                
            # Load class names
            dataset_path = "Philippine Medicinal Plant Leaf Dataset"
            if os.path.exists(dataset_path):
                self.class_names = sorted([name for name in os.listdir(dataset_path) 
                                         if os.path.isdir(os.path.join(dataset_path, name))])
                print(f"Found {len(self.class_names)} plant classes")
            else:
                self.class_names = [f"Class_{i}" for i in range(40)]
                print("Using fallback class names")
            
            messagebox.showinfo("Model Loaded", f"Successfully loaded model with {len(self.class_names)} classes")
                
        except Exception as e:
            print(f"Error loading model: {e}")
            messagebox.showerror("Error", f"Failed to load model: {str(e)}")
    
    def open_camera(self):
        """Open camera window"""
        try:
            self.camera = cv2.VideoCapture(0)
            if not self.camera.isOpened():
                messagebox.showerror("Camera Error", "Cannot access camera")
                return
            
            self.camera_running = True
            self.camera_frame.deiconify()
            self.update_camera()
            
        except Exception as e:
            messagebox.showerror("Camera Error", f"Failed to open camera: {str(e)}")
    
    def update_camera(self):
        """Update camera feed"""
        if self.camera_running and self.camera:
            ret, frame = self.camera.read()
            if ret:
                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame_resized = cv2.resize(frame_rgb, (640, 480))
                pil_image = Image.fromarray(frame_resized)
                photo = ImageTk.PhotoImage(pil_image)
                self.camera_label.configure(image=photo)
                self.camera_label.image = photo
                
            self.camera_frame.after(10, self.update_camera)
    
    def capture_image(self):
        """Capture image from camera"""
        if self.camera:
            ret, frame = self.camera.read()
            if ret:
                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                self.current_image = Image.fromarray(frame_rgb)
                self.display_image(self.current_image)
                self.close_camera()
                self.predict_plant()
    
    def close_camera(self):
        """Close camera"""
        self.camera_running = False
        if self.camera:
            self.camera.release()
            self.camera = None
        self.camera_frame.withdraw()
    
    def open_gallery(self):
        """Open file dialog to select image"""
        file_path = filedialog.askopenfilename(
            title="Select Plant Leaf Image",
            filetypes=[("Image files", "*.jpg *.jpeg *.png *.bmp *.tiff")]
        )
        
        if file_path:
            try:
                print(f"Loading image from: {file_path}")
                self.current_image = Image.open(file_path)
                print(f"Image loaded successfully: {self.current_image.size}")
                self.display_image(self.current_image)
                print("Calling predict_plant...")
                self.predict_plant()
            except Exception as e:
                print(f"Gallery error: {e}")
                messagebox.showerror("Error", f"Failed to load image: {str(e)}")
    
    def display_image(self, pil_image):
        """Display image with enhanced styling - responsive sizing"""
        # Create enhanced image display with responsive dimensions
        display_image = pil_image.copy()
        
        # Responsive image size based on screen resolution
        if self.is_large_screen:
            max_size = (500, 400)
            border_width = 25
        else:
            max_size = (400, 300)
            border_width = 20
            
        display_image.thumbnail(max_size, Image.Resampling.LANCZOS)
        
        # Add subtle border effect
        bordered_image = Image.new('RGB', 
                                  (display_image.width + border_width, display_image.height + border_width), 
                                  color=self.colors['accent'])
        bordered_image.paste(display_image, (border_width//2, border_width//2))
        
        photo = ImageTk.PhotoImage(bordered_image)
        self.image_label.configure(image=photo, text="")
        self.image_label.image = photo
        
        self.root.after_idle(self._update_scroll_region)
    
    def analyze_image_content(self, img_array):
        """Enhanced image analysis for better leaf detection including colored leaves"""
        try:
            img_cv = (img_array * 255).astype(np.uint8)
            gray = cv2.cvtColor(img_cv, cv2.COLOR_RGB2GRAY)
            
            # Face detection (keep existing logic)
            face_detected = False
            total_face_area = 0
            
            try:
                face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
                faces = face_cascade.detectMultiScale(gray, scaleFactor=1.05, minNeighbors=3, minSize=(30, 30))
                
                if len(faces) > 0:
                    total_face_area = sum([w*h for (x,y,w,h) in faces])
                    face_detected = True
                    print(f"Frontal faces detected: {len(faces)}, total area: {total_face_area}")
                
                eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
                eyes = eye_cascade.detectMultiScale(gray, scaleFactor=1.05, minNeighbors=3, minSize=(10, 10))
                if len(eyes) >= 2:
                    print(f"Eyes detected: {len(eyes)}")
                    if len(faces) > 0:
                        return "human_face", 98.0
                        
            except Exception as e:
                print(f"Face detection error: {e}")
            
            if face_detected and total_face_area > 1000:
                return "human_face", 95.0
            
            # Enhanced vegetation analysis for all leaf colors
            hsv = cv2.cvtColor(img_cv, cv2.COLOR_RGB2HSV)
            
            # Multiple vegetation color ranges (not just green)
            vegetation_ranges = [
                # Traditional green vegetation
                ([35, 40, 40], [85, 255, 255]),      # Standard green
                ([25, 30, 30], [95, 255, 255]),      # Extended green range
                
                # Purple/Maroon leaves (like Coleus)
                ([120, 40, 30], [160, 255, 200]),    # Purple range
                ([0, 40, 30], [20, 255, 200]),       # Red-purple range
                ([160, 40, 30], [180, 255, 200]),    # Magenta range
                
                # Brown/Reddish leaves
                ([0, 30, 40], [25, 200, 150]),       # Brown-red
                ([10, 50, 50], [25, 255, 180]),      # Reddish brown
                
                # Yellow/Orange leaves  
                ([15, 50, 50], [35, 255, 255]),      # Yellow-orange
                
                # Dark colored leaves
                ([0, 20, 20], [180, 100, 80]),       # Very dark leaves
            ]
            
            vegetation_ratios = []
            total_vegetation_mask = np.zeros(gray.shape, dtype=np.uint8)
            
            for i, (lower, upper) in enumerate(vegetation_ranges):
                lower_veg = np.array(lower)
                upper_veg = np.array(upper)
                veg_mask = cv2.inRange(hsv, lower_veg, upper_veg)
                total_vegetation_mask = cv2.bitwise_or(total_vegetation_mask, veg_mask)
                
                veg_ratio = np.sum(veg_mask > 0) / (img_cv.shape[0] * img_cv.shape[1])
                vegetation_ratios.append(veg_ratio)
                print(f"Vegetation range {i}: {veg_ratio:.3f}")
            
            total_vegetation_ratio = np.sum(total_vegetation_mask > 0) / (img_cv.shape[0] * img_cv.shape[1])
            max_vegetation_ratio = max(vegetation_ratios) if vegetation_ratios else 0
            
            # Skin detection (existing logic)
            skin_ranges = [
                ([0, 20, 50], [20, 255, 255]),
                ([0, 10, 40], [25, 200, 255]),
                ([0, 30, 60], [18, 255, 255]),
                ([160, 10, 40], [180, 200, 255])
            ]
            
            skin_ratios = []
            total_skin_mask = np.zeros(gray.shape, dtype=np.uint8)
            
            for i, (lower, upper) in enumerate(skin_ranges):
                lower_skin = np.array(lower)
                upper_skin = np.array(upper)
                skin_mask = cv2.inRange(hsv, lower_skin, upper_skin)
                total_skin_mask = cv2.bitwise_or(total_skin_mask, skin_mask)
                
                skin_ratio = np.sum(skin_mask > 0) / (img_cv.shape[0] * img_cv.shape[1])
                skin_ratios.append(skin_ratio)
            
            total_skin_ratio = np.sum(total_skin_mask > 0) / (img_cv.shape[0] * img_cv.shape[1])
            
            # Enhanced leaf structure detection
            edges = cv2.Canny(gray, 50, 150)
            edge_density = np.sum(edges > 0) / (edges.shape[0] * edges.shape[1])
            
            # Texture analysis for leaf detection
            blur = cv2.GaussianBlur(gray, (5, 5), 0)
            laplacian_var = cv2.Laplacian(blur, cv2.CV_64F).var()
            
            # Contour analysis for leaf-like shapes
            contours, _ = cv2.findContours(total_vegetation_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            
            # Find the largest contour (presumably the leaf)
            if contours:
                largest_contour = max(contours, key=cv2.contourArea)
                contour_area = cv2.contourArea(largest_contour)
                total_area = img_cv.shape[0] * img_cv.shape[1]
                contour_ratio = contour_area / total_area
                
                # Check if contour is leaf-like (not too circular, has some complexity)
                perimeter = cv2.arcLength(largest_contour, True)
                if perimeter > 0:
                    circularity = 4 * np.pi * contour_area / (perimeter * perimeter)
                else:
                    circularity = 1.0
            else:
                contour_ratio = 0
                circularity = 1.0
            
            # Color analysis
            rgb_mean = np.mean(img_cv, axis=(0,1))
            color_variance = np.var(img_cv, axis=(0,1))
            total_color_variance = np.sum(color_variance)
            
            # Dark features analysis
            lab = cv2.cvtColor(img_cv, cv2.COLOR_RGB2LAB)
            l_channel = lab[:,:,0]
            very_dark = cv2.inRange(l_channel, 0, 50)
            dark = cv2.inRange(l_channel, 0, 80)
            
            very_dark_ratio = np.sum(very_dark > 0) / (very_dark.shape[0] * very_dark.shape[1])
            dark_ratio = np.sum(dark > 0) / (dark.shape[0] * dark.shape[1])
            
            print(f"Enhanced Analysis:")
            print(f"  Vegetation: {total_vegetation_ratio:.3f} (max: {max_vegetation_ratio:.3f})")
            print(f"  Skin: {total_skin_ratio:.3f}")
            print(f"  Edge density: {edge_density:.3f}")
            print(f"  Laplacian var: {laplacian_var:.1f}")
            print(f"  Contour ratio: {contour_ratio:.3f}")
            print(f"  Circularity: {circularity:.3f}")
            print(f"  Color variance: {total_color_variance:.1f}")
            print(f"  Dark ratio: {dark_ratio:.3f}")
            
            # Enhanced decision logic for leaf detection
            
            # Human detection (priority check)
            if face_detected and total_skin_ratio > 0.05:
                return "human_face", 98.0
            
            if total_skin_ratio > 0.08 and max_vegetation_ratio < 0.3 and dark_ratio > 0.03:
                return "human_skin", 90.0
            
            if total_skin_ratio > 0.05 and very_dark_ratio > 0.02 and max_vegetation_ratio < 0.25:
                return "human_skin", 85.0
            
            # Enhanced leaf detection scoring system
            leaf_score = 0
            confidence = 50.0
            
            # Vegetation color presence (any plant color)
            if total_vegetation_ratio > 0.4:
                leaf_score += 3
                confidence += 20
            elif total_vegetation_ratio > 0.2:
                leaf_score += 2
                confidence += 10
            elif max_vegetation_ratio > 0.15:
                leaf_score += 1
                confidence += 5
            
            # Edge structure (leaves have distinctive edges)
            if edge_density > 0.06:
                leaf_score += 2
                confidence += 15
            elif edge_density > 0.03:
                leaf_score += 1
                confidence += 8
            
            # Texture complexity (leaves have natural texture)
            if laplacian_var > 150:
                leaf_score += 2
                confidence += 10
            elif laplacian_var > 80:
                leaf_score += 1
                confidence += 5
            
            # Shape analysis (leaf-like contour)
            if contour_ratio > 0.3:
                leaf_score += 2
                confidence += 15
                # Check if shape is not too circular (leaves aren't perfect circles)
                if 0.2 < circularity < 0.8:
                    leaf_score += 1
                    confidence += 10
            elif contour_ratio > 0.15:
                leaf_score += 1
                confidence += 8
            
            # Color complexity (natural leaves have color variation)
            if total_color_variance > 500:
                leaf_score += 1
                confidence += 5
            
            # Penalize human-like features
            if total_skin_ratio > 0.03:
                leaf_score -= 2
                confidence -= 15
            
            print(f"Leaf detection score: {leaf_score}/11, confidence: {confidence:.1f}%")
            
            # Final classification
            if leaf_score >= 6:
                return "possible_leaf", min(95.0, confidence)
            elif leaf_score >= 4:
                return "possible_leaf", min(80.0, confidence)
            elif leaf_score >= 2:
                return "vegetation", min(70.0, confidence)
            elif total_vegetation_ratio > 0.1 or max_vegetation_ratio > 0.05:
                return "vegetation", min(60.0, confidence)
            else:
                return "other_object", 40.0
                
        except Exception as e:
            print(f"Error in enhanced image analysis: {e}")
            import traceback
            traceback.print_exc()
            return "unknown", 0.0

    def check_leaf_detection(self, predictions, img_array):
        """Enhanced leaf detection with human detection priority"""
        max_confidence = np.max(predictions) * 100
        
        content_type, content_confidence = self.analyze_image_content(img_array)
        
        print(f"Content analysis: {content_type} with confidence {content_confidence}")
        print(f"AI model confidence: {max_confidence:.1f}%")
        
        if content_type in ["human_face", "human_skin"]:
            print("Human detected - overriding AI prediction")
            return False, "human_detected", content_type
        
        if content_type == "other_object" and max_confidence < 60:
            return False, "non_vegetation", content_type
        
        if content_type in ["possible_leaf", "vegetation"]:
            if max_confidence < 20:
                return False, "unclear_leaf", content_type
            elif max_confidence < 40:
                return True, "uncertain", content_type
            else:
                return True, "confident", content_type
        else:
            if max_confidence < self.min_confidence_threshold:
                return False, "very_low", content_type
            elif max_confidence < self.uncertainty_threshold + 10:
                return True, "uncertain", content_type
            else:
                return True, "confident", content_type
    
    def predict_plant(self):
        """Predict plant species using the Keras model with leaf detection"""
        print("predict_plant function called")
        
        if self.model is None:
            print("No model found")
            messagebox.showwarning("Model Error", "Model not loaded")
            return
        
        if not self.current_image:
            print("No current image")
            messagebox.showwarning("Image Error", "No image selected")
            return
        
        print(f"Model loaded: {self.model is not None}")
        print(f"Image size: {self.current_image.size}")
        print(f"Class names count: {len(self.class_names)}")
        
        try:
            print("Starting preprocessing...")
            img_resized = self.current_image.resize((224, 224))
            
            if img_resized.mode != 'RGB':
                img_resized = img_resized.convert('RGB')
                print(f"Converted image from {self.current_image.mode} to RGB")
            
            img_array = np.array(img_resized, dtype=np.float32)
            print(f"Image array before normalization - Min: {img_array.min()}, Max: {img_array.max()}, Mean: {img_array.mean():.2f}")
            
            img_array_normalized = img_array / 255.0
            img_array_normalized = np.expand_dims(img_array_normalized, axis=0)
            
            print(f"Image array shape: {img_array_normalized.shape}")
            print(f"Normalized image - Min: {img_array_normalized.min():.3f}, Max: {img_array_normalized.max():.3f}, Mean: {img_array_normalized.mean():.3f}")
            print("Making prediction...")
            
            predictions = self.model.predict(img_array_normalized, verbose=0)
            
            print(f"Predictions shape: {predictions.shape}")
            print(f"Raw predictions (first 10): {predictions[0][:10]}")
            
            pred_std = np.std(predictions[0])
            pred_range = np.max(predictions[0]) - np.min(predictions[0])
            print(f"Prediction variance - Std: {pred_std:.6f}, Range: {pred_range:.6f}")
            
            is_leaf_detected, detection_quality, object_type = self.check_leaf_detection(predictions[0], img_array_normalized[0])
            
            predicted_class_idx = np.argmax(predictions[0])
            confidence = np.max(predictions[0]) * 100
            predicted_class = self.class_names[predicted_class_idx]
            
            print(f"Leaf detected: {is_leaf_detected}, Quality: {detection_quality}, Object type: {object_type}")
            print(f"Predicted class index: {predicted_class_idx}")
            print(f"Predicted class: {predicted_class}")
            print(f"Confidence: {confidence:.2f}%")
            
            top5_indices = np.argsort(predictions[0])[-5:][::-1]
            print("Top 5 predictions:")
            for i, idx in enumerate(top5_indices):
                print(f"  {i+1}. {self.class_names[idx]}: {predictions[0][idx]*100:.2f}%")
            
            if not is_leaf_detected:
                self.display_no_leaf_detected(confidence, predictions[0], detection_quality, object_type)
            else:
                self.display_results(predicted_class, confidence, predictions[0], detection_quality)
            
        except Exception as e:
            print(f"Prediction error: {e}")
            import traceback
            traceback.print_exc()
            messagebox.showerror("Prediction Error", f"Failed to predict: {str(e)}")
    
    def create_modern_card(self, parent, title, icon, color_scheme):
        """Create a modern card widget - properly sized and spaced"""
        # Create container frame for centering with better spacing
        card_container = tk.Frame(parent, bg=self.colors['white'])
        card_container.pack(fill='x', pady=(0, 25))
        
        # Better proportional max width based on screen size
        max_card_width = 600 if self.is_large_screen else 550
        min_side_padding = 60  # Minimum padding from edges
        
        card = tk.Frame(card_container, bg=self.colors['white'], relief='solid', bd=1)
        
        # Configure card to center within container with proper spacing
        def center_card():
            card_container.update_idletasks()
            container_width = card_container.winfo_width()
            
            if container_width > (max_card_width + (min_side_padding * 2)):
                # Center the card with calculated padding
                side_padding = max(min_side_padding, (container_width - max_card_width) // 2)
                card.pack(pady=15, padx=side_padding, fill='x')
            else:
                # Use minimum padding if container is smaller
                card.pack(pady=15, padx=min_side_padding, fill='x')
        
        card_container.after(1, center_card)
        
        # Card header with better proportions
        header_height = 55 if self.is_large_screen else 50
        header = tk.Frame(card, bg=color_scheme, height=header_height)
        header.pack(fill='x')
        header.pack_propagate(False)
        
        # Header font sizing
        header_font_size = 16 if self.is_large_screen else 15
        
        header_label = tk.Label(header, text=f"{icon} {title}", 
                               font=('Arial', header_font_size, 'bold'), 
                               fg='white', bg=color_scheme)
        header_label.pack(expand=True)
        
        # Card content with better spacing
        content_padding_y = 25 if self.is_large_screen else 20
        content_padding_x = 30 if self.is_large_screen else 25
        
        content = tk.Frame(card, bg=self.colors['white'])
        content.pack(pady=content_padding_y, padx=content_padding_x, fill='both', expand=True)
        
        return card, content
    
    def display_no_leaf_detected(self, confidence, all_predictions, detection_quality, object_type):
        """Display enhanced no-leaf detection message - with proper centering"""
        for widget in self.results_frame.winfo_children():
            widget.destroy()
        
        # Create centered results container
        results_container = tk.Frame(self.results_frame, bg=self.colors['white'])
        results_container.pack(expand=True, fill='both')
        
        # Determine styling based on object type
        if object_type == "human_face":
            icon, title = "🚫", "No Leaf Detection"
            message = "Object is not identified as a medicinal plant leaf."
            color = self.colors['danger']
        elif object_type == "human_skin":
            icon, title = "🚫", "No Leaf Detection"
            message = "Object is not identified as a medicinal plant leaf."
            color = self.colors['danger']
        elif object_type == "other_object":
            icon, title = "🚫", "No Leaf Detection"
            message = "Object is not identified as a medicinal plant leaf."
            color = self.colors['warning']
        elif object_type == "vegetation":
            icon, title = "🌱", "Vegetation Detected"
            message = "Plant material detected, but not from our medicinal plant database."
            color = self.colors['warning']
        else:
            icon, title = "🚫", "No Clear Leaf Detected"
            message = "Object is not identified as a medicinal plant leaf."
            color = self.colors['danger']
        
        # Create main result card
        result_card, result_content = self.create_modern_card(
            results_container, "Detection Results", "⚠️", color
        )
        
        # Status message
        status_frame = tk.Frame(result_content, bg='#FFF3E0', relief='solid', bd=1, pady=20)
        status_frame.pack(fill='x', pady=(0, 20))
        
        tk.Label(status_frame, text=f"{icon} {title}", 
                font=('Arial', 18, 'bold'), fg=color, bg='#FFF3E0').pack(pady=(15, 5))
        
        tk.Label(status_frame, text=message, 
                font=('Arial', 13), fg=color, bg='#FFF3E0').pack(pady=(0, 15))
        
        # Suggestions card
        suggestions_card, suggestions_content = self.create_modern_card(
            results_container, "Suggestions for Better Results", "💡", self.colors['info']
        )
        
        # Customize suggestions based on detection type
        if object_type in ["human_face", "human_skin"]:
            suggestions = [
                "Focus the camera only on a plant leaf",
                "Remove any people or body parts from the frame",
                "Take a close-up photo of just the leaf",
                "Ensure good lighting on the plant leaf"
            ]
        elif object_type == "other_object":
            suggestions = [
                "Make sure you're photographing a plant leaf",
                "Remove any non-plant objects from the frame",
                "Use natural lighting for better plant recognition",
                "Ensure the leaf is the main subject of the photo"
            ]
        else:
            suggestions = [
                "Use a clear, well-lit photo of a single leaf",
                "Ensure the leaf fills most of the image frame",
                "Try a different angle or lighting condition",
                "Make sure the leaf is from a Philippine medicinal plant"
            ]
        
        for i, suggestion in enumerate(suggestions, 1):
            suggestion_frame = tk.Frame(suggestions_content, bg=self.colors['white'])
            suggestion_frame.pack(fill='x', pady=5)
            
            tk.Label(suggestion_frame, text=f"{i}.", 
                    font=('Arial', 12, 'bold'), fg=self.colors['info'], 
                    bg=self.colors['white']).pack(side='left', padx=(0, 10))
            
            tk.Label(suggestion_frame, text=suggestion, 
                    font=('Arial', 12), fg=self.colors['text_primary'], 
                    bg=self.colors['white'], anchor='w').pack(side='left', fill='x', expand=True)
        
        # Show top predictions for vegetation
        if object_type in ["vegetation", "unclear_leaf", "possible_leaf"]:
            top_card, top_content = self.create_modern_card(
                results_container, "Closest Matches (Very Low Confidence)", "📊", self.colors['warning']
            )
            
            top3_indices = np.argsort(all_predictions)[-3:][::-1]
            
            for i, idx in enumerate(top3_indices):
                class_name = self.class_names[idx].replace('_', ' ')
                conf = all_predictions[idx] * 100
                
                pred_frame = tk.Frame(top_content, bg='#FFF8E1', relief='solid', bd=1)
                pred_frame.pack(fill='x', pady=3, padx=5)
                
                tk.Label(pred_frame, text=f"{i+1}. {class_name}: {conf:.1f}%", 
                        font=('Arial', 12), fg=self.colors['warning'], 
                        bg='#FFF8E1', pady=8, padx=15, anchor='w').pack(fill='x')
        
        self.root.after_idle(self._update_scroll_region)
    
    def display_results(self, predicted_class, confidence, all_predictions, detection_quality):
        """Display enhanced prediction results - responsive sizing"""
        for widget in self.results_frame.winfo_children():
            widget.destroy()
        
        # Determine styling based on confidence
        if detection_quality == "confident":
            header_color = self.colors['secondary']
            header_icon = "✅"
            header_title = "Plant Identified Successfully"
        else:
            header_color = self.colors['warning']
            header_icon = "⚠️"
            header_title = "Plant Identified (Low Confidence)"
        
        # Main result card
        result_card, result_content = self.create_modern_card(
            self.results_frame, header_title, header_icon, header_color
        )
        
        # Responsive font sizes
        warning_font_size = 16 if self.is_large_screen else 14
        plant_title_font_size = 16 if self.is_large_screen else 14
        plant_name_font_size = 20 if self.is_large_screen else 18
        confidence_font_size = 16 if self.is_large_screen else 14
        
        # Confidence warning for uncertain detections
        if detection_quality == "uncertain":
            warning_frame = tk.Frame(result_content, bg='#FFF8E1', relief='solid', bd=1)
            warning_frame.pack(fill='x', pady=(0, 20))
            
            tk.Label(warning_frame, text="⚠️ Low Confidence Detection", 
                    font=('Arial', warning_font_size, 'bold'), fg=self.colors['warning'], 
                    bg='#FFF8E1').pack(pady=(15, 5))
            
            tk.Label(warning_frame, 
                    text="The leaf may be partially obscured or at an unusual angle.\nConsider taking a clearer photo for better results.", 
                    font=('Arial', 12 if self.is_large_screen else 11), fg=self.colors['warning'], 
                    bg='#FFF8E1', justify='center').pack(pady=(0, 15))
        
        # Plant identification result with better spacing
        id_frame = tk.Frame(result_content, bg=self.colors['light'], relief='solid', bd=1)
        id_frame.pack(fill='x', pady=(0, 25))
        
        # Plant name with better spacing and sizing
        title_padding = 25 if self.is_large_screen else 20
        
        tk.Label(id_frame, text="🍃 Identified Plant", 
                font=('Arial', plant_title_font_size, 'bold'), fg=self.colors['primary'], 
                bg=self.colors['light']).pack(pady=(title_padding, 8))
        
        tk.Label(id_frame, text=predicted_class.replace('_', ' '), 
                font=('Arial', plant_name_font_size, 'bold'), fg=self.colors['primary'], 
                bg=self.colors['light']).pack(pady=(0, 15))
        
        # Confidence display with better proportions
        conf_padding = 30 if self.is_large_screen else 25
        
        conf_frame = tk.Frame(id_frame, bg=self.colors['light'])
        conf_frame.pack(pady=(0, title_padding), padx=conf_padding, fill='x')
        
        tk.Label(conf_frame, text=f"Confidence: {confidence:.1f}%", 
                font=('Arial', confidence_font_size), fg=self.colors['primary'], 
                bg=self.colors['light']).pack(pady=(0, 12))
        
        # Modern progress bar with better sizing
        progress_height = 28 if self.is_large_screen else 24
        progress_margin = 15 if self.is_large_screen else 10
        
        progress_container = tk.Frame(conf_frame, bg=self.colors['gray_medium'], 
                                     height=progress_height, relief='solid', bd=1)
        progress_container.pack(pady=(0, 0), padx=progress_margin, fill='x')
        progress_container.pack_propagate(False)
        
        # Progress fill
        if confidence >= 70:
            fill_color = self.colors['secondary']
        elif confidence >= 40:
            fill_color = self.colors['warning']
        else:
            fill_color = self.colors['danger']
        
        def create_progress():
            progress_container.update()
            actual_width = progress_container.winfo_width()
            fill_width_actual = max(1, int((confidence / 100) * actual_width))
            
            progress_fill = tk.Frame(progress_container, bg=fill_color, height=progress_height-2)
            progress_fill.place(x=1, y=1, width=fill_width_actual)
        
        progress_container.after(1, create_progress)
        
        # Top 3 predictions card
        top_card, top_content = self.create_modern_card(
            self.results_frame, "Top 3 Predictions", "📊", self.colors['info']
        )
        
        top3_indices = np.argsort(all_predictions)[-3:][::-1]
        
        # Responsive prediction card sizing
        rank_width = 50 if self.is_large_screen else 40
        rank_font_size = 16 if self.is_large_screen else 14
        pred_name_font_size = 14 if self.is_large_screen else 12
        pred_conf_font_size = 12 if self.is_large_screen else 10
        pred_padding = 12 if self.is_large_screen else 10
        
        for i, idx in enumerate(top3_indices):
            class_name = self.class_names[idx].replace('_', ' ')
            conf = all_predictions[idx] * 100
            
            pred_frame = tk.Frame(top_content, bg='#E3F2FD', relief='solid', bd=1)
            pred_frame.pack(fill='x', pady=3)
            
            # Rank badge
            rank_frame = tk.Frame(pred_frame, bg=self.colors['info'], width=rank_width)
            rank_frame.pack(side='left', fill='y')
            rank_frame.pack_propagate(False)
            
            tk.Label(rank_frame, text=str(i+1), font=('Arial', rank_font_size, 'bold'), 
                    fg='white', bg=self.colors['info']).pack(expand=True)
            
            # Prediction details
            detail_frame = tk.Frame(pred_frame, bg='#E3F2FD')
            detail_frame.pack(side='left', fill='both', expand=True, padx=15, pady=pred_padding)
            
            tk.Label(detail_frame, text=class_name, font=('Arial', pred_name_font_size, 'bold'), 
                    fg=self.colors['text_primary'], bg='#E3F2FD', anchor='w').pack(fill='x')
            
            tk.Label(detail_frame, text=f"Confidence: {conf:.1f}%", font=('Arial', pred_conf_font_size), 
                    fg=self.colors['text_secondary'], bg='#E3F2FD', anchor='w').pack(fill='x')
        
        # Plant information card (if available)
        if predicted_class in self.plant_info:
            info = self.plant_info[predicted_class]
            
            info_card, info_content = self.create_modern_card(
                self.results_frame, "Medicinal Information", "🌿", self.colors['secondary']
            )
            
            # Responsive disclaimer sizing
            disclaimer_font_size = 14 if self.is_large_screen else 12
            
            # Confidence disclaimer for uncertain detections
            if detection_quality == "uncertain":
                disclaimer_frame = tk.Frame(info_content, bg='#FFEBEE', relief='solid', bd=1)
                disclaimer_frame.pack(fill='x', pady=(0, 15))
                
                tk.Label(disclaimer_frame, 
                        text="⚠️ Please verify identification before medicinal use due to low confidence", 
                        font=('Arial', disclaimer_font_size, 'bold'), fg=self.colors['danger'], 
                        bg='#FFEBEE', pady=15).pack()
            
            # Scientific information with responsive fonts
            info_label_font = 14 if self.is_large_screen else 12
            info_value_font = 13 if self.is_large_screen else 11
            info_spacing = 10 if self.is_large_screen else 8
            value_indent = 25 if self.is_large_screen else 20
            
            info_details = [
                ("🔬 Scientific Name", info['scientific_name']),
                ("🏷️ Common Names", ', '.join(info['common_names'])),
                ("💊 Medicinal Uses", ', '.join(info['medicinal_uses'])),
                ("📝 Preparation", info['preparation'])
            ]
            
            for label, value in info_details:
                detail_frame = tk.Frame(info_content, bg=self.colors['white'])
                detail_frame.pack(fill='x', pady=info_spacing)
                
                tk.Label(detail_frame, text=label, font=('Arial', info_label_font, 'bold'), 
                        fg=self.colors['primary'], bg=self.colors['white'], 
                        anchor='w').pack(fill='x')
                
                # Responsive text wrapping
                wrap_length = 600 if self.is_large_screen else 500
                
                tk.Label(detail_frame, text=value, font=('Arial', info_value_font), 
                        fg=self.colors['text_primary'], bg=self.colors['white'], 
                        anchor='w', wraplength=wrap_length, justify='left').pack(fill='x', padx=(value_indent, 0))
        
        self.root.after_idle(self._update_scroll_region)
    
    def _on_mousewheel(self, event):
        """Handle mouse wheel scrolling"""
        self.canvas.yview_scroll(int(-1*(event.delta/120)), "units")
    
    def _on_canvas_configure(self, event):
        """Handle canvas resize to center content - improved centering"""
        canvas_width = event.width
        
        # Update the scrollable frame width to match canvas
        self.canvas.itemconfig(self.canvas_window, width=canvas_width)
        
        # Center content within the canvas
        frame_width = self.scrollable_frame.winfo_reqwidth()
        
        if canvas_width > frame_width:
            x_offset = (canvas_width - frame_width) // 2
        else:
            x_offset = 0
            
        self.canvas.coords(self.canvas_window, x_offset, 0)
    
    def _update_scroll_region(self):
        """Update the scroll region to fit all content - improved centering"""
        self.canvas.update_idletasks()
        self.canvas.configure(scrollregion=self.canvas.bbox("all"))
        
        # Re-center the content after updating scroll region
        canvas_width = self.canvas.winfo_width()
        frame_width = self.scrollable_frame.winfo_reqwidth()
        
        if canvas_width > frame_width:
            x_offset = (canvas_width - frame_width) // 2
        else:
            x_offset = 0
            
        self.canvas.coords(self.canvas_window, x_offset, 0)

def main():
    root = tk.Tk()
    app = PlantIdentifierApp(root)
    root.mainloop()

if __name__ == "__main__":
    main()

Loading model from: philippine_medicinal_plants_cnn_new.keras
Model loaded successfully!
Using fallback class names
Loading image from: C:/Users/donat/Desktop/School Folder/Year 3/Sem 3/Deep Learning/Project/Demo/Tester/Adelfa.jpg
Image loaded successfully: (3000, 4000)
Calling predict_plant...
predict_plant function called
Model loaded: True
Image size: (3000, 4000)
Class names count: 40
Starting preprocessing...
Image array before normalization - Min: 33.0, Max: 218.0, Mean: 186.60
Image array shape: (1, 224, 224, 3)
Normalized image - Min: 0.129, Max: 0.855, Mean: 0.732
Making prediction...
Predictions shape: (1, 40)
Raw predictions (first 10): [3.3385680e-18 2.0464886e-15 7.8023186e-14 1.1585586e-19 1.2276760e-13
 2.6759514e-14 5.3114688e-15 6.6875812e-17 8.0431868e-19 4.5282167e-19]
Prediction variance - Std: 0.156125, Range: 1.000000
Face detection error: module 'cv2' has no attribute 'data'
Vegetation range 0: 0.059
Vegetation range 1: 0.062
Vegetation range 2: 0.001
Vegetation 